threading do scale and ended up using most of the cores, 

parsing is slow still


75232/75232 [==============================] - 144s 2ms/step - loss: 0.2044 - acc: 0.9252 - val_loss: 0.1596 - val_acc: 0.9404




Need to improve the tokenizer


In [19]:
%matplotlib inline

"""

This example shows how to use an LSTM sentiment classification model trained using Keras in spaCy. spaCy splits the document into sentences, and each sentence is classified using the LSTM. The scores for the sentences are then aggregated to give the document score. This kind of hierarchical model is quite difficult in "pure" Keras or Tensorflow, but it's very effective. The Keras example on this dataset performs quite poorly, because it cuts off the documents so that they're a fixed size. This hurts review accuracy a lot, because people often summarise their rating in the final sentence

Prerequisites:
spacy download en_vectors_web_lg
pip install keras==2.0.9

Compatible with: spaCy v2.0.0+

"""

import plac
import random
import pathlib
import cytoolz
import numpy
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import thinc.extra.datasets
from spacy.compat import pickle
import spacy


In [20]:
#!python -m spacy download en_vectors_web_lg

In [63]:
%%time
nlp=spacy.load('en_vectors_web_lg')


CPU times: user 12.7 s, sys: 768 ms, total: 13.4 s
Wall time: 13.4 s


In [64]:
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [65]:
#nlp.vocab.prune_vectors(10000)

In [49]:
### pipeline in the data
"""
return: list of sentences, for each doc
"""
def get_labelled_sentences(docs, doc_labels):
    labels = []
    sentences = []
    for doc, y in zip(docs, doc_labels):
        for sent in doc.sents:
            sentences.append(sent)
            labels.append(y)
    return sentences, numpy.asarray(labels, dtype='int32')

"""
input:list of docs
Xs: doc by token matrix max length, each entry is the ID to the data
"""
def get_features(docs, max_length):
    docs = list(docs)
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            ##rever to word vector
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs

#lstm_shape: {'nr_hidden': 64, 'max_length': 100, 'nr_class': 1},nr_hidden: reduce the number of activation?
def train(train_texts, train_labels, dev_texts, dev_labels,
          lstm_shape, lstm_settings, lstm_optimizer, batch_size=100,
          nb_epoch=5, by_sentence=True):
    #print("Loading spaCy")
    #nlp = spacy.load('en_core_web_lg')
    
    embeddings = get_embeddings(nlp.vocab)
    #embedgigs is vocab.vectors.data
    print ('embeddings:',embeddings)
    print ('lstm_shape:',lstm_shape)
    print ('lstm_settings:',lstm_settings)
    model = compile_lstm(embeddings, lstm_shape, lstm_settings)
    print ('model:',model)
    print("Parsing texts...")
    train_docs = list(nlp.pipe(train_texts))
    dev_docs = list(nlp.pipe(dev_texts))
    if by_sentence:
        train_docs, train_labels = get_labelled_sentences(train_docs, train_labels)
        dev_docs, dev_labels = get_labelled_sentences(dev_docs, dev_labels)

    train_X = get_features(train_docs, lstm_shape['max_length'])
    dev_X = get_features(dev_docs, lstm_shape['max_length'])
    model.fit(train_X, train_labels, validation_data=(dev_X, dev_labels),
              nb_epoch=nb_epoch, batch_size=batch_size)
    return model

"""
input: doc by tokens ID, in the form of setence. 

TimeDistributed: 

embeddings: 
"""
def compile_lstm(embeddings, shape, settings):
    model = Sequential()
    ###embedding layer, 
    model.add(
        Embedding(
            embeddings.shape[0],
            embeddings.shape[1],
            input_length=shape['max_length'],
            trainable=False,
            weights=[embeddings],
            mask_zero=True
        )
    )
    ## TimeDistributed: wrapper applies a layer to every temporal slice of an input.
    ## dense/ fully connected: a NN layer
    ## dense, nr_hidden, reduce the number of units
    ## 
    model.add(TimeDistributed(Dense(shape['nr_hidden'], use_bias=False)))
    model.add(Bidirectional(LSTM(shape['nr_hidden'],
                                 recurrent_dropout=settings['dropout'],
                                 dropout=settings['dropout'])))
    model.add(Dense(shape['nr_class'], activation='sigmoid'))
    #lr: learning rate, binary_crossentropy
    model.compile(optimizer=Adam(lr=settings['lr']), loss='binary_crossentropy',
		  metrics=['accuracy'])
    return model


def get_embeddings(vocab):
    return vocab.vectors.data



### read in SRA

In [25]:
#inS_dir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'
import pandas as pd

In [26]:
inS_dir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'

sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'

srsS=pd.read_pickle(inS_dir)


technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)

tmpDf=technical_meta_data_df.drop_duplicates('Sample')

technical_meta_data_df=tmpDf.sample(n=tmpDf.shape[0],random_state=0)

#technical_meta_data_df.head()

n=5000
subsampledSrs=technical_meta_data_df.groupby('Study').head(n=n)['Sample']

In [27]:
#srsS.value_counts

In [28]:
myAttribs=['cell type','sex','age','genotype','treatment','SCIENTIFIC_NAME']
m=srsS.index.get_level_values(0).isin(subsampledSrs.values)
m1=srsS.index.get_level_values(1).isin(myAttribs)
srsS_subS=srsS[m&m1]

In [29]:
#srsS_subS.value_counts()

In [30]:
from sklearn import preprocessing

In [31]:
le = preprocessing.LabelEncoder()
le.fit(myAttribs)


LabelEncoder()

In [32]:
myDf=srsS_subS.reset_index()
myDf.columns=['srs','attrib','sentence']

In [33]:
from sklearn import model_selection

In [34]:
inTrainTestDf=myDf.sample(n=myDf.shape[0]).groupby('attrib').head(n=20000)

train_df,test_df=model_selection.train_test_split(inTrainTestDf)

In [36]:
model_dir=None
train_dir=None
dev_dir=None
is_runtime=False
nr_hidden=64
max_length=50#, # Shape
dropout=0.5
learn_rate=0.001#, # General NN config
nb_epoch=1#
batch_size=100
nr_examples=-1  # Training params

In [37]:
nr_classes=len(le.classes_)

In [38]:
import keras

In [39]:
#y_train = keras.utils.to_categorical(y_train, num_classes)
keras.utils.to_categorical(le.transform(train_df.attrib.tolist()))

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [40]:
lstm_shape={'nr_hidden': 64, 'max_length': max_length, 'nr_class': nr_classes}


In [50]:
%%time
lstm=train(train_df.sentence.tolist(),keras.utils.to_categorical(le.transform(train_df.attrib.tolist())),
     test_df.sentence.tolist(),keras.utils.to_categorical(le.transform(test_df.attrib.tolist())),
     {'nr_hidden': nr_hidden, 'max_length': max_length, 'nr_class': nr_classes},
             {'dropout': dropout, 'lr': learn_rate},
             {},
             nb_epoch=nb_epoch, batch_size=batch_size)

embeddings: [[ 0.042175   0.34913   -0.32363   ...  0.22986   -0.30662    0.3878   ]
 [-0.63354   -0.1503    -0.36161   ...  0.26216   -0.12094    0.0038262]
 [-0.65276    0.23873   -0.23325   ... -0.42636    0.48578   -0.28969  ]
 ...
 [-0.51423   -0.23612   -0.035289  ... -0.20719   -0.60762    1.0162   ]
 [ 0.32269   -0.41217    0.044183  ...  0.29696    0.32911   -0.22805  ]
 [ 0.9066    -1.1523    -1.2483    ...  0.17064    0.56711    0.13522  ]]
lstm_shape: {'nr_hidden': 64, 'max_length': 50, 'nr_class': 6}
lstm_settings: {'dropout': 0.5, 'lr': 0.001}
model: <keras.engine.sequential.Sequential object at 0x2afc6361b470>
Parsing texts...


/cellar/users/btsui/Data/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 90957 samples, validate on 30293 samples
Epoch 1/1
90957/90957 [==============================] - 123s 1ms/step - loss: 0.1760 - acc: 0.9399 - val_loss: 0.1312 - val_acc: 0.9543
CPU times: user 19min 36s, sys: 12min 40s, total: 32min 17s
Wall time: 2min 20s


In [51]:
#nlp = spacy.load('en_core_web_lg')
#nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [24]:
m1=srsS.index.get_level_values(1)=='SCIENTIFIC_NAME'
m2=srsS.values=='Mus musculus'
scientifcS=srsS[m1&m2]

In [56]:
#test_df

In [58]:
#lstm.predict()

In [25]:
%%time
myAmbiguosAttribs=['TITLE']
m=srsS.index.get_level_values(0).isin(subsampledSrs.values)
m1=srsS.index.get_level_values(1).isin(myAmbiguosAttribs)
m_species=srsS.index.get_level_values(0).isin(scientifcS.index.get_level_values(0))
srsS_subS=srsS[m&m1&m_species]

CPU times: user 7.22 s, sys: 788 ms, total: 8.01 s
Wall time: 8 s


In [26]:
len(srsS_subS)

230628

In [27]:
textDf_ambigous=srsS_subS.reset_index()
textDf_ambigous.columns=['srs','attrib','sentence']

In [28]:
subDf=textDf_ambigous.sample(n=100)
amb_text=subDf.sentence.str.replace('_',' ').tolist()

In [29]:

#embeddings = get_embeddings(nlp.vocab)

In [30]:
amb_docs=nlp.pipe(amb_text)

In [31]:
M=get_features(amb_docs,max_length=lstm_shape['max_length'])

In [32]:
#M

In [33]:
#lstm.predict_proba(M)

In [34]:
scoreDf=pd.DataFrame( lstm.predict_proba(M),index=subDf.set_index(['srs','attrib','sentence']).index
            ,columns=le.classes_)

In [35]:
scoreDf['predicted']=le.inverse_transform(lstm.predict_classes(M))

/cellar/users/btsui/Data/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
#srsS.loc['SRS1645209']

In [37]:
myKeys=nlp.vocab.vectors

SyntaxError: invalid syntax (<ipython-input-37-3335b9dca393>, line 1)

In [ ]:
#nlp.vocab.strings

In [ ]:
#nlp.vocab.vectors.shape

In [ ]:
scoreDf.sort_values('cell type',ascending=False)

In [ ]:
#scoreDf.idxmax(axis=1).value_counts()

In [ ]:
#le.transform(test_df.sentence.tolist())

In [ ]:
### feed titles for testing. 

In [ ]:
"""
    print("Loading spaCy")
    nlp = spacy.load('en_core_web_lg')
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    embeddings = get_embeddings(nlp.vocab)
    #embedgigs is vocab.vectors.data
    print ('embeddings:',embeddings)
    print ('lstm_shape:',lstm_shape)
    print ('lstm_settings:',lstm_settings)
    model = compile_lstm(embeddings, lstm_shape, lstm_settings)
    print ('model:',model)
    print("Parsing texts...")
    train_docs = list(nlp.pipe(train_texts))
    dev_docs = list(nlp.pipe(dev_texts))
    if by_sentence:
        train_docs, train_labels = get_labelled_sentences(train_docs, train_labels)
        dev_docs, dev_labels = get_labelled_sentences(dev_docs, dev_labels)

    train_X = get_features(train_docs, lstm_shape['max_length'])
    dev_X = get_features(dev_docs, lstm_shape['max_length'])
    model.fit(train_X, train_labels, validation_data=(dev_X, dev_labels),
              nb_epoch=nb_epoch, batch_size=batch_size)
"""

In [ ]:
embeddings = get_embeddings(nlp.vocab)

In [4]:
train_X

NameError: name 'train_X' is not defined

In [ ]:
"""
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or direc
libcudnn.so.7:

conda install -c anaconda cudatoolkit
conda install -c anaconda cudnn
conda uninstall glibc -c asmeurer
/lib64/libc.so.6:
"""

In [1]:
!grep sex *.ipynb

getSrsCounts.ipynb:       "      <th>sex</th>\n",
getSrsCounts.ipynb:       "sex                                        163806\n",
getSrsCounts.ipynb:      "       'isolate', 'sex', 'env_biome', 'env_feature', 'env_material',\n",
getSrsCounts.ipynb:    "sns.pairplot(tsneDf.reset_index()[['attrib',0,1]],hue='attrib',hue_order=['sex','gender','tissue','cell type'])"
keras_on_sra_data.ipynb:    "myAttribs=['cell type','sex','age','genotype','treatment','SCIENTIFIC_NAME','phenotype']\n",
keras_on_sra_data.ipynb:       "       'sex', 'treatment'], dtype='<U15')"
keras_on_sra_data.ipynb:      "image/png": "...\n",
keras_on_sra_data_old.ipynb:    "myAttribs=['cell type','sex','age','genotype','treatment','SCIENTIFIC_NAME']\n",
read_embedding_matrix.ipynb:      "image/png": "...\n",
